In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import math
import random

In [2]:
def phi(x, r):
    return r * x * (1 - x)

def get_window(r):
    return (max(0.0, (r - 1) / (2 * r)), (r + 1) / (2 * r))

In [3]:
def simple_iterations(epsilon, r):
    (left_border, right_border) = get_window(r)
    iterations = 0
    
    cur_x = random.uniform(left_border, right_border)
    next_x = phi(cur_x, r) 
    
    while (abs(cur_x - next_x) > epsilon):
        cur_x = next_x
        next_x = phi(cur_x, r)
        iterations += 1
        if (iterations > 1000):
            return (-100, 1000)
        
    return (next_x, iterations)

In [7]:
roots = []
roots.append(0)
roots.append(0)

for i in range(1, 10**4):
    cur_r = random.uniform(0.01, 0.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
print(roots)

[9999, 0]


In [13]:
roots = []
roots.append(0)
roots.append(0)

undefined_roots = []

for i in range(1, 10**4):
    cur_r = random.uniform(1.01, 2.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
    else:
        undefined_roots.append(cur_r)
print(roots) 
print(undefined_roots)

[0, 9971]
[2.98660928791283, 2.9852267444855283, 2.986563964634926, 2.9859407243837603, 2.984741052826788, 2.9862713522733433, 2.989473907035208, 2.9863877149182976, 2.986812265624107, 2.987624380557224, 2.9873591449788517, 2.986616312791573, 2.9886856683505902, 2.98597881329001, 2.9852818936087164, 2.9881142099200773, 2.9876351051581524, 2.985030173728821, 2.987792496945506, 2.988703314965284, 2.986133445191535, 2.988597097120576, 2.986904608491538, 2.986361174793017, 2.9879626733488767, 2.986182568223801, 2.989660945121712, 2.9888679846366877]


In [12]:
count = 0

for i in range(1, 10**4):
    cur_r = random.uniform(3.01, 10.0)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    if (answer < -50):
        count += 1
print(count)

9999


x = r * x * (1 - x)
phi(x) = r * x * (1 - x)
phi'(x) = r - 2rx
чтобы решение сходилось к корню, первое приближение должно лежать в окрестности корня, удовлетворяющей условию |phi'(x)| <= 1
|phi'(x)| <= 1 => max(0.0, (r - 1) / (2 * r)) < x < (r + 1) / (2 * r)

x0 = 0
x1 = 1 - 1/r

Рассмотрим phi'(x0) = r
Чтобы в окрестности корня выполнялось |phi'(x)| < 1, должно быть |r| < 1 => r < 1
Значит, при 0 < r < 1 сходимость к x0 = 0 возможна

Рассмотрим phi'(x1) = 2 - r
|phi'(x)| < 1 => |2 - r| < 1 => 1 < r < 3
При 1 < r < 3 сходимость к x1 = 1 - 1/r возможна

Заметим, что области сходимости к x0 (0; 1) и к x1 (1; 3) не пересекаются